# Введение
---

**Цель анализа**: провести первичную оценку качества набора данных о физико-химических свойствах веществ, выявить основные проблемы в данных и подготовить их к дальнейшему анализу.

**Источник:** [Kaggle — Physical and Chemical Properties of Substances](https://www.kaggle.com/datasets/ivanyakovlevg/physical-and-chemical-properties-of-substances/data)

**Содержание:**

Набор данных содержит информацию о различных химических веществах, включая их физические и термодинамические свойства.
Каждая запись соответствует одному веществу и включает следующие параметры:

* Химическая формула и название
* Молекулярная масса
* Температуры плавления и кипения
* Плотность
* Теплоёмкость, энтальпия, энтропия
* Критическая температура и давление
* Другие физико-химические параметры, используемые в термодинамическом моделировании и вычислительной химии

Этот датасет подходит для:
* Оценки качества и полноты данных
* Анализа корреляций и признаков
* Построения предсказательных моделей для оценки неизвестных свойств веществ

План работы:
1. Загрузка и первичный осмотр
2. Оценка полноты данных
3. Анализ выбросов и аномалий
4. Итоговые выводы

# **Важные особенности рассматриваемого датасета!**
Нюанс данного датасета заключается в его **физико-химической природе**.

Это означает, что к анализу данных следует подходить не в духе классического статистического анализа, а с учётом законов физики и химии.

Сильные отклонения отдельных значений от средних не всегда являются выбросами -
в ряде случаев это естественные особенности веществ, отражающие реальные физические свойства.

Однако, некоторые значения могут быть признаны аномальными не по статистическим причинам,
а по физическим законам.

Например, температура не может быть ниже 0 Кельвинов, даже если статистически это не выброс.

# Блок импортов

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

## Загрузка данных и первичный осмотр ##

Используем скрипт ```data_loader.py``` для загрузки данных и преобразовании в ```parquet```

In [2]:
%run ../src/data_loader.py
df = pd.read_parquet('clean_data.parquet')

Файл уже есть, читаю локальную копию.
name                     object
formula                  object
CAS                      object
smiles                   object
InChI                    object
InChIKey                 object
molecular_weight        float64
melting_point_K         float64
boiling_point_K         float64
heat_of_fusion          float64
heat_of_vaporization    float64
critical_temperature    float64
critical_pressure       float64
flash_point             float64
logP                    float64
dtype: object
                                 name        formula        CAS  \
0                             ammonia            H3N  7664-41-7   
1  1,4-benzodioxane-2-carboxylic acid         C9H8O4  3663-80-7   
2                           acetylene           C2H2    74-86-2   
3              adenosine triphosphate  C10H16N5O13P3    56-65-5   
4                     rhodizonic acid         C6H2O6   118-76-3   

                                              smiles  \
0        

Данные успешно загружены и преобразованы в ```parquet```. Командами ```raw_data.dtypes.head(15)``` и
```raw_data.head(5)``` выведены первые столбцы и строки и типы данных.

# Настройки визуализации

In [3]:
sns.set_theme(style="whitegrid")
custom_palette = sns.color_palette("Spectral", as_cmap=False) # Используем широко применяемую в химии Spectral Palette из Origin Lab

numeric_cols = df.select_dtypes(include=np.number).columns

## Оценка целостности и полноты данных

Проверим данные на наличие пропущенных значений и полных дубликатов. Это критически важный шаг для оценки качества данных.

In [4]:
# Считаем количество пропусков в каждом столбце
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Создаем DataFrame для наглядного отображения пропусков
missing_info = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
})
missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values(by='Percentage (%)', ascending=False)

# Визуализация пропущенных значений
fig_missing = px.bar(
    missing_info,
    x=missing_info.index,
    y='Percentage (%)',
    title='Процент пропущенных значений по столбцам',
    labels={'index': 'Столбец', 'Percentage (%)': 'Процент пропусков (%)'}
)
fig_missing.update_layout(xaxis_tickangle=-45) # Наклоняем подписи
fig_missing.show()

# Проверяем наличие полных дубликатов
num_duplicates = df.duplicated().sum()
print(f"\nНайдено полных дубликатов строк: {num_duplicates}")


Найдено полных дубликатов строк: 0


### Выводы и метрики по целостности

* **Метрика: Количество столбцов с пропусками:** ```15```
* **Метрика: Процент пропусков для самого проблемного столбца:** ```93.7```
* **Метрика: Количество дубликатов:** `0`

**Вывод:** анализ показал наличие пропусков в нескольких столбцах. Наиболее проблемным является столбец `flash_point`, где отсутствует `93.7%` данных. Это может потребовать либо удаления строк, либо использования стратегий для заполнения пропусков. Полных дубликатов обнаружено не было.

## Оценка выбросов и аномалий

Изучим распределения данных, чтобы найти значения, которые сильно выделяются из общей картины. Для этого воспользуемся описательными статистиками для числовых признаков и подсчетом значений для булевых.

In [5]:
# Получаем описательные статистики для всех числовых столбцов
# 'display()' используется для красивого вывода в Jupyter
print("Описательные статистики для числовых признаков:")
display(df.describe())

print("\nВизуализация распределений числовых признаков (с разделением по фасетам)")

# Выбираем только числовые столбцы из .describe()
numeric_cols = df.describe().columns
df_melted = df[numeric_cols].melt()

fig_box = px.box(
    df_melted,
    y='value',               # Значение по-прежнему на Y
    facet_row='variable',    # !!! КЛЮЧЕВОЕ ИЗМЕНЕНИЕ: каждый признак - своя строка
    title='Распределение числовых признаков (у каждого свой масштаб)',
    labels={'value': 'Значение'},
    points='outliers'        # По-прежнему показываем выбросы
)

# Это ГЛАВНАЯ команда:
# Мы "отвязываем" оси Y друг от друга, 
# чтобы у каждого графика-строки был свой собственный масштаб.
fig_box.update_yaxes(matches=None, showticklabels=True)

# Увеличим высоту, чтобы графики не слипались
fig_box.update_layout(height=2000, width=800) 

fig_box.show()

Описательные статистики для числовых признаков:


,molecular_weight,melting_point_K,boiling_point_K,heat_of_fusion,heat_of_vaporization,critical_temperature,critical_pressure,flash_point,logP
count,4343.000000,4343.000000,3634.000000,3.356000e+03,1.823000e+03,3519.000000,3.501000e+03,273.000000,709.000000
mean,269.160696,514.716704,811.599397,1.424487e+05,4.532314e+05,1106.093084,5.595430e+06,338.622960,2.097278
std,179.075599,400.066559,546.277999,1.318577e+05,1.331007e+06,4384.442863,3.378248e+07,69.082691,2.175028
min,1.007940,0.950000,4.223807,2.554609e+02,0.000000e+00,-81884.985000,1.062812e+05,185.751749,-3.690000
25%,157.336690,337.265000,502.425000,1.048341e+05,2.175428e+05,750.140760,1.916937e+06,285.150000,0.510000
50%,244.285740,432.150000,715.830000,1.299734e+05,2.968804e+05,966.760120,2.826327e+06,334.150000,2.050000
75%,337.411450,530.150000,935.177500,1.560281e+05,4.088440e+05,1183.086050,4.360679e+06,383.150000,3.610000
max,3354.070500,7449.440000,8574.340000,3.438407e+06,3.584615e+07,222858.230000,8.421600e+08,549.816670,9.360000



Визуализация распределений числовых признаков (с разделением по фасетам)


# Поиск физических и логических аномалий

In [6]:
print("Ищем значения, которые физически невозможны (например, T < 0K) или нелогичны.\n")

# Определяем столбцы, где температура ДОЛЖНА быть > 0 K
temp_cols = [
    'melting_point_K', 
    'boiling_point_K', 
    'critical_temperature', 
    'flash_point'
]

has_anomalies = False

# Проверка 1: Температуры < 0 K
print("Проверка 1: температуры < абсолютного нуля (0 K)")
for col in temp_cols:
    # Находим строки, где значение < 0 и оно не является NaN
    anomalies = df[(df[col] < 0) & (df[col].notna())]
    if not anomalies.empty:
        print(f"Аномалии в '{col}' ({len(anomalies)})")
        display(anomalies[['name', 'formula', col]])
        has_anomalies = True

if not has_anomalies:
    print("Проверка T < 0 K: физически невозможных температур не найдено")

# Проверка 2: молекулярная масса <= 0
print("\nПроверка 2: молекулярная масса <= 0")
mw_anomalies = df[(df['molecular_weight'] <= 0) & (df['molecular_weight'].notna())]
if not mw_anomalies.empty:
    print(f"Аномалии 'molecular_weight' ({len(mw_anomalies)})")
    display(mw_anomalies[['name', 'formula', 'molecular_weight']])
    has_anomalies = True
else:
    print("Проверка Mw <= 0: аномальных значений массы не найдено")

# Проверка 3: температура плавления > температуры кипения
print("\nПроверка 3: T плавления > T кипения")
logic_anomalies = df[
    (df['melting_point_K'] > df['boiling_point_K']) & 
    (df['melting_point_K'].notna()) & 
    (df['boiling_point_K'].notna())
]
if not logic_anomalies.empty:
    display(logic_anomalies[['name', 'formula', 'melting_point_K', 'boiling_point_K']])
    has_anomalies = True
else:
    print("Проверка T_melt > T_boil: Логических аномалий не найдено.")

if not has_anomalies:
    print("\nЯвных физических или логических ошибок в данных не обнаружено.")

Ищем значения, которые физически невозможны (например, T < 0K) или нелогичны.

Проверка 1: температуры < абсолютного нуля (0 K)
Аномалии в 'critical_temperature' (14)


,name,formula,critical_temperature
214,vancomycin,C66H75Cl2N9O24,-4406.6773
478,einecs 215-807-5,C143H230N42O37S7,-896.4172
600,tannic acid,C76H52O46,-4007.8763
618,ac1l1igg,C47H75NO17,-67885.2360
619,amphozone,C47H73NO17,-81884.9850
699,colistin,C52H98N16O13,-17852.9090
785,deacetylchitin,C56H103N9O39,-1072.1010
1216,palytoxin,C129H223N3O54,-347.7808
1396,glycogen,C24H42O21,-19681.6610
1555,digitonin,C56H92O29,-1940.5202



Проверка 2: молекулярная масса <= 0
Проверка Mw <= 0: аномальных значений массы не найдено

Проверка 3: T плавления > T кипения


,name,formula,melting_point_K,boiling_point_K
2,acetylene,C2H2,192.40,189.00035
13,carbon dioxide,CO2,216.65,194.67000
21,nsc94017,C10H12N5O6P,492.65,329.27000
34,ac1laxt3,C5H5N5O,712.65,688.07000
70,cyclooctasulfur,S8,761.94,615.00000
...,...,...,...,...
4173,"(2r,3r)-2-(3,4-dihydroxyphenyl)-2,3-dihydro-3,...",C15H12O8,1048.56,997.75000
4232,ambazone,C8H11N7S,466.15,433.15000
4245,carbonyl dibromide,CBr2O,410.15,337.15000
4254,n-butylcyclohexanamine,C10H21N,481.45,480.15000



# Удаление физических и логических аномалий


In [7]:
print("Удаление физических и логических аномалий")

# Запоминаем исходный размер датафрейма
initial_rows = len(df)
print(f"Исходное количество строк: {initial_rows}")

# Создаем чистый датафрейм, который мы будем использовать дальше
# Мы не удаляем строки с NaN, а только те, где есть явные ошибки
df_clean = df.copy()


temp_cols = [
    'melting_point_K', 
    'boiling_point_K', 
    'critical_temperature', 
    'flash_point'
]
anomaly_indices_temp = set()
for col in temp_cols:
    indices = df_clean[(df_clean[col] < 0) & (df_clean[col].notna())].index
    anomaly_indices_temp.update(indices)

if anomaly_indices_temp:
    df_clean = df_clean.drop(index=list(anomaly_indices_temp))

anomaly_indices_mw = df_clean[
    (df_clean['molecular_weight'] <= 0) & (df_clean['molecular_weight'].notna())
].index

if not anomaly_indices_mw.empty:
    df_clean = df_clean.drop(index=anomaly_indices_mw)

anomaly_indices_logic = df_clean[
    (df_clean['melting_point_K'] > df_clean['boiling_point_K']) & 
    (df_clean['melting_point_K'].notna()) & 
    (df_clean['boiling_point_K'].notna())
].index

if not anomaly_indices_logic.empty:
    df_clean = df_clean.drop(index=anomaly_indices_logic)

final_rows = len(df_clean)
removed_rows = initial_rows - final_rows

print(f"Итоговое количество строк: {final_rows}")
print(f"Всего удалено аномальных строк: {removed_rows}")

Удаление физических и логических аномалий
Исходное количество строк: 4343
Итоговое количество строк: 4184
Всего удалено аномальных строк: 159


Для анализа достоверности и качества очистки данных удобно использовать известные физико-химические закономерности. Если очищенные данные им не следуют, это указывает на сохраняющиеся ошибки или пропуски.

Наиболее фундаментальная закономерность, связанная с силами межмолекулярного взаимодействия, гласит:

**Температура кипения в целом растет с увеличением молекулярной массы**

Это правило особенно явно прослеживается в гомологических рядах, например, у углеводородов.

Проверим эту закономерность на наших очищенных данных (df_clean). Мы будем использовать логарифмический масштаб для осей, чтобы лучше визуализировать широкий диапазон значений и выделить точки, которые сильно отклоняются от основного тренда (потенциально интересные аномалии).

In [8]:
# Отфильтруем, чтобы избежать проблем с log(0)
df_scatter = df_clean.dropna(subset=['boiling_point_K', 'molecular_weight'])

fig_scatter = px.scatter(
    df_scatter,
    x='molecular_weight',
    y='boiling_point_K',
    # Раскрашиваем точки по ключевой категориальной переменной
    color='is_hydrocarbon', 
    hover_data=['name', 'formula', 'CAS'], # Показываем информацию о веществе при наведении
    log_x=True, # Используем логарифмическую шкалу для X (молекулярная масса)
    log_y=True, # Используем логарифмическую шкалу для Y (T кипения)
    title='Зависимость температуры кипения от молекулярной массы (логарифмический масштаб)',
    labels={'molecular_weight': 'Молекулярная масса (log)', 'boiling_point_K': 'T кипения, K (log)'}
)

fig_scatter.update_layout(title_font_size=20)
fig_scatter.show()

**Исследуемые данные соблюдают данный закон, что говорит об их достоверности и пригодности для дальнейшего изучения**

# Тепловая карта корреляций

Карта корреляций позволит более глобально посмотреть на исследуемые признаки, подтвердить закономерности и заменить сильно коррелирующие одним (PCA) для удобства в дальнейшей работе и потенциале использования в ML.

In [9]:
# Считаем корреляцию только для числовых столбцов
corr_matrix = df_clean[numeric_cols].corr()

fig_heatmap = px.imshow(
    corr_matrix,
    text_auto=True,  # Показать значения
    aspect="auto",
    color_continuous_scale='Viridis', # Используем единую палитру
    title='Тепловая карта корреляций числовых признаков'
)
fig_heatmap.update_layout(height=800, width=800, title_font_size=20)
fig_heatmap.show()

**Наиболее сильные положительные корреляции (ρ≈0.8 и выше) соответствуют ожидаемым физическим зависимостям, что подтверждает общую достоверность большей части данных.**

Фазовые переходы (ρ≈0.84): Самая сильная корреляция наблюдается между T 
кипения (boiling_point_K) и T плавления (melting_point_K) (ρ=0.8399). 
Это логично: вещества, которые плавятся при высоких температурах, как правило, и кипят при высоких.

**Выявление слабых и потенциально интересных связей (средняя и низкая корреляция)**

Корреляция между H плавления (heat_of_fusion) и H испарения (heat_of_vaporization) составляет всего ρ≈0.54. Это указывает на то, что в датасете присутствуют как вещества, требующие много энергии для плавления (например, с сильной кристаллической решеткой), так и вещества, которые требуют много энергии для кипения (например, с сильными водородными связями). **Нет единого доминирующего типа вещества.**

# Динамическая матрица рассеяния
Удостоверившись в адекватности данных, проведём небольшое исследование.

Одним из важных параметров молекул является **липофильность** - способность к растворению в жирах, маслах и подобных им неполярных растворителях. Динамическая матрица рассеяния (Scatter Plot) исследует связь между олекулярной массой (Mw) и липофильностью (logP), с дополнительной классификацией по химическому классу. Динамический элемент (слайдер) позволяет фильтровать данные по категории superclass, что делает анализ чрезвычайно глубоким.

In [10]:
import plotly.express as px
import pandas as pd

# Заменяем NaN в числовых столбцах на медиану (чтобы график не падал)
df_plot = df.copy()
for col in ['boiling_point_K', 'melting_point_K', 'heat_of_vaporization']:
    if col in df_plot.columns:
        df_plot[col] = df_plot[col].fillna(df_plot[col].median())

# Фильтруем по реалистичным значениям
df_plot = df_plot[df_plot['molecular_weight'] < 1000]  # отсечем экстремальные значения

# Создаём динамический график
fig = px.scatter(
    df_plot,
    x='molecular_weight',
    y='logP',
    color='class',
    size='boiling_point_K',
    hover_data=['melting_point_K', 'heat_of_vaporization'],
    animation_frame='superclass', 
    color_discrete_sequence=px.colors.sequential.Sunset,
    title="Связь массы и липофильности соединений с динамикой по химическим классам"
)

fig.update_layout(
    template='plotly_white',
    font=dict(size=14),
    title_font=dict(size=20),
    legend_title_text='Класс соединения',
    xaxis_title="Молекулярная масса (g/mol)",
    yaxis_title="logP (липофильность)"
)

fig.show()

Исследование зависимости (Параметр-Параметр) по конкретному классу соединений полезно по следующей причине. 

**Вся химия построена на принципе: "Структура определяет свойство". Одно и то же правило не может работать для неорганических солей и сложных органических полимеров.**

Вывод: при построении одной общей модели, например, для предсказания logP по Mw для всего датасета, она будет неточной, поскольку пытается усреднить различные химические законы.

Динамический график показывает, что необходимо:

1) Либо включить признак superclass (или class) как сильный категориальный фактор в общую модель.

2) Либо разбить задачу на части и построить отдельные модели (например, одну для "Organic oxygen compounds", другую для "Inorganic salts"), поскольку зависимость Mw от logP внутри каждой группы будет гораздо более линейной и предсказуемой.

#### Выводы по числовым признакам

Анализ описательных статистик позволяет сделать следующие наблюдения:

* **Столбец `molecular_weight`:** среднее значение ([mean]) близко к медиане ([50%]), что говорит о достаточно симметричном распределении. Минимальное и максимальное значения выглядят правдоподобно.
* **Столбец `critical_temperature`:** присутствуют сильно отрицательные значения в `min`, что физически невозможно для температуры. Это явная аномалия или ошибка в данных.

#### Анализ булевых и категориальных признаков

In [11]:
# Анализируем распределение булевых признаков
bool_cols = [c for c in df_clean.columns if c.startswith("is_")]
for col in bool_cols:
    print(f"Распределение для столбца '{col}':")
    print(df_clean[col].value_counts(normalize=True)) # normalize=True показывает долю

# Проверяем уникальность идентификатора CAS 
print(f"Всего записей: {len(df_clean)}")
print(f"Уникальных CAS: {df_clean['CAS'].nunique()}")
if len(df_clean) == df_clean['CAS'].nunique():
    print("Идентификатор 'CAS' является уникальным для каждой строки.")
else:
    print("В 'CAS' есть дублирующиеся значения!")

Распределение для столбца 'is_organic':
is_organic
True     0.828155
False    0.171845
Name: proportion, dtype: float64
Распределение для столбца 'is_radionuclide':
is_radionuclide
False    1.0
Name: proportion, dtype: float64
Распределение для столбца 'is_hydrocarbon':
is_hydrocarbon
False    0.968451
True     0.031549
Name: proportion, dtype: float64
Распределение для столбца 'is_alkane':
is_alkane
False    0.989723
True     0.010277
Name: proportion, dtype: float64
Распределение для столбца 'is_cycloalkane':
is_cycloalkane
False    0.996893
True     0.003107
Name: proportion, dtype: float64
Распределение для столбца 'is_branched_alkane':
is_branched_alkane
False    0.999283
True     0.000717
Name: proportion, dtype: float64
Распределение для столбца 'is_alkene':
is_alkene
False    0.810468
True     0.189532
Name: proportion, dtype: float64
Распределение для столбца 'is_alkyne':
is_alkyne
False    0.990201
True     0.009799
Name: proportion, dtype: float64
Распределение для столбца '

#### Выводы по булевым и категориальным признакам

* **Булевы признаки:** в данных преобладают органические соединения ([83]%). Радионуклиды отсутствуют. Значительная часть веществ - углеводороды.
* **Идентификатор `cas_number`:** проверка показала, что значения CAS дублируются, что может быть ошибкой в данных и требует правки/удаления.

## 5. Заключение

---
Проведенный разведочный анализ данных (EDA) позволил критически оценить качество и структуру предоставленного набора данных

### Основные выводы:

* **Структура и Типы:** данные имеют четкую структуру. Категориальные (химические) и числовые признаки корректны после первичной обработки.
* **Полнота:** обнаружены **значительные пропуски** в ключевых числовых столбцах (до 80% в некоторых признаках), что является основной проблемой для потенциального моделирования.
* **Корректность:** выявлены и **удалены явные физические и логические аномалии** (например, $\text{T} < 0 \text{K}$).
* **Достоверность:** основные физико-химические законы (например, $\text{T}_{кипения} \sim \text{MW}$) подтверждены на очищенных данных, что свидетельствует об их высокой достоверности в целом.
* **Моделирование:** показано, что для точного моделирования необходимо **учитывать категориальный признак химического класса** (`superclass`), поскольку он является сильным предиктором свойств (например, $\text{logP}$).
